In [ ]:
import os
import getpass
import warnings
from pprint import pprint
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageLayoutAnalysisLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import AIMessage, HumanMessage
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
from langchain_chroma import Chroma
import retreive  # Make sure this is your custom module
# Suppress warnings
warnings.filterwarnings("ignore")
# Load environment variables
load_dotenv()
# Initialize OpenAI client
client = OpenAI(
    api_key=os.environ["UPSTAGE_API_KEY"],
    base_url="https://api.upstage.ai/v1/solar"
)
# Initialize Upstage LLM
llm = ChatUpstage(streaming=True)
# Define the system prompt
SYSTEM_PROMPT = """당신은 글로벌 경제 분석 전문가입니다. 정치, 경제, 사회적 이벤트가 세계 경제에 미치는 영향을 분석하고
    다양한 시나리오를 예측하여 제시합니다.
    분석 방법:
    1. 제시된 이벤트의 직접적인 영향 분석
    2. 연관된 산업과 지역에 대한 파급 효과 분석
    3. 리스크 요인 식별
    답변 형식:
    1. 제목
    2. 사건 배경
    3. 시나리오 분석
    4. 결론 및 시사점
    5. 참조 링크
    요구사항:
    1.이해하기 쉬운 용어로 설명
    2.참조 링크는 url 마크다운 형태로 표기."""
def generate_response(user_input):
    """Generate response using the Solar API"""
    claimed_facts = retreive.extracted_claimed_facts(user_input)
    relevant_context = retreive.search_context(user_input, claimed_facts)
    chat_result = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"user:{user_input} report: {relevant_context}"}
        ],
    )
    return chat_result.choices[0].message.content
def chat(message, history):
    """Combined chat function that uses both LangChain and Solar API"""
    # Convert history to LangChain format
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    # Generate response using Solar API
    response = generate_response(message)
    return response
# Create Gradio interface
with gr.Blocks() as demo:
    chatbot = gr.ChatInterface(
        chat,
        examples=[
            "트럼프가 대선에서 승리하면 한국경제에 어떤 영향을 미칠까?",
            "한국은행의 기준금리 인상은 어떤 영향을 미칠까?",
            "유가 급등에 의한 영향은 어떤게 있을까?",
        ],
        title="Solar Chatbot",
        description="Upstage Solar Chatbot with Economic Analysis",
    )
    chatbot.chatbot.height = 300
if __name__ == "__main__":
    demo.launch()
